# Hyperparameter tuning

This notebook focuses on tuning the hyperparameters of an initial network to find a better model on the training set. 
The starting point is a 4 layered feed forward neural network with 120, 60, 25 and 10 neurons per layer.

## Random search
To reduce computation a random search is preferred to a complete grid search.
A total of 100 models are tested.

In [1]:
import sys
sys.path.append("..")
from src.model import NeuralNetwork
import numpy as np
np.random.seed(1)

def print_results(results):
    print("Best: %f using %s\n" % (results.best_score_, results.best_params_))
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean, stdev, param in sorted(zip(means, stds, params), key=lambda x : -x[0])[:3]:
        print("%f (%f) with: %r" % (mean, stdev, param))
    print("...")

In [4]:
k = 2
neurons = [(180, 80 + i*k, 46 + j*k, 10) for i in (-2,-1,0,1,2) for j in (-2,-1,0,1,2)]
learning_rate = [0.001, 0.01, 0.1, 0.5]
momentum = [0.0, 0.01, 0.1, 1]
epochs = [60, 80, 100]
batch_size = [32, 64]

param_dist = dict(neurons=neurons, 
                  learning_rate=learning_rate,
                  momentum=momentum, 
                  epochs=epochs, 
                  batch_size=batch_size)

extended_results = NeuralNetwork.optimize_model(method="random", 
                                                param_grid=param_dist,
                                                dataset_path="../data/processed/extended/train_pca.csv", 
                                                iterations=50)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


In [5]:
print_results(extended_results)

Best: 0.630796 using {'neurons': (180, 80, 44, 10), 'momentum': 0.1, 'learning_rate': 0.1, 'epochs': 100, 'batch_size': 32}

0.630796 (0.036974) with: {'neurons': (180, 80, 44, 10), 'momentum': 0.1, 'learning_rate': 0.1, 'epochs': 100, 'batch_size': 32}
0.626793 (0.051451) with: {'neurons': (180, 80, 42, 10), 'momentum': 0.01, 'learning_rate': 0.1, 'epochs': 60, 'batch_size': 32}
...


In [6]:
param_dist["neurons"] = [(102, 45 + i*k, 30 + j*k, 10) for i in (-2,-1,0,1,2) for j in (-2,-1,0,1,2)]

pca_results = NeuralNetwork.optimize_model(method="random", 
                                           param_grid=param_dist,
                                           dataset_path="../data/processed/extended/train_pca.csv", 
                                           iterations=50)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


In [7]:
print_results(pca_results)

Best: 0.625463 using {'neurons': (102, 45, 28, 10), 'momentum': 0.1, 'learning_rate': 0.1, 'epochs': 100, 'batch_size': 32}

0.625463 (0.041491) with: {'neurons': (102, 45, 28, 10), 'momentum': 0.1, 'learning_rate': 0.1, 'epochs': 100, 'batch_size': 32}
0.617906 (0.040060) with: {'neurons': (102, 45, 26, 10), 'momentum': 0.01, 'learning_rate': 0.1, 'epochs': 60, 'batch_size': 32}
...


## Final model
The best model found is the following

In [10]:

best_param = extended_results.best_params_
best_param

{'neurons': (180, 80, 44, 10),
 'momentum': 0.1,
 'learning_rate': 0.1,
 'epochs': 100,
 'batch_size': 32}

## Test results

The final model is evaluated on the test set to determine performances.

In [12]:
p = extended_results.best_params_

model = NeuralNetwork.create_model(neurons=p["neurons"],
                                   learning_rate=p["learning_rate"], 
                                   momentum=p["momentum"])

from src.data import Dataset
from sklearn.utils import class_weight

d = Dataset(dataset_path="../data/processed/extended/train_pca.csv", 
            test_size=0)

from keras.callbacks import EarlyStopping
stopper = EarlyStopping(monitor='accuracy', patience=3, verbose=1)
fit_params = dict(callbacks=[stopper])

x, y = d.get_splits()
class_weights = class_weight.compute_class_weight('balanced', np.unique(y), y)
weights_dict = dict(zip(np.unique(y), class_weights))

model.fit(x, y, class_weight=weights_dict, epochs=p["epochs"], batch_size=p["batch_size"], verbose=0, **fit_params)

Epoch 00017: early stopping


In [13]:
from src.utils import show_accuracy_loss

accuracy, loss = show_accuracy_loss(model, scaling="pca", test_dataset_path="../data/processed/extended")

7/7 [==============================] - 0s 2ms/step - loss: 1.8852 - accuracy: 0.6858

Accuracy:
	Mean: 0.671506917476654 
	Standard deviation: 0.05384217473067788

Loss:
	Mean: 2.1888545989990233 
	Standard deviation: 0.5279860807927551
